# Data Preparation

<p style="line-height:1.75;font-size:16px">
As we've talked about earlier, researchers spend a lot of their time processing and preparing the data prior to modeling. In most cases, this process is lengthy, tedious and non-trivial but it has a very big impact on our final results and therefore needs to be addressed.<br>
In this notebook, we will talk about various techiques of dealing with common issues that you may encounter in your work. This is by no means a comprehensive guide of all the possible issues and solutions but rather a set of tools that you can add to the ones we've already talked about earlier (e.g. data scaling).

## Handling Missing Values

### What is it?

<p style="line-height:1.75;font-size:16px">
Missing values are probably the most common issues you'll encounter when dealing with data. As the name suggests, i merely means that certain features in our dataset have no values for some or all records. There are many reasons why values can be missing from a dataset - thess reasons might also affect the way we choose to handle these missing values.

### Why is it an issue?

<p style="line-height:1.75;font-size:16px">
From a technical perspective, some models do not know how to deal with missing (n/a, nan) values and will just return an error when we try to give them data with missing values. Even if the model does know how to deal with missing values, we must still ask ourselves - should we let it?<Br> When it boils down to it, a model needs a numerical value to work with so it will need to assign <u>some</u> value to the missing value. The number 0 is most likely to be assigned but it doesn't always makes sense. For instance, what if we have a feature measuring a person's height in cm, having a value of 0 is probably not a good idea.<br>  
It is far better to have control of how we fill in these missing values rather then letting the model choose an arbitrary value which might not make a lot of sense.

### Possible solutions

#### Dropping rows

<p style="line-height:1.75;font-size:16px">
If row (record) is missing a lot of values or missing a specific value that is critical for our prediction, we might choose to drop this row altogether. Using this solution might lead to a large loss of data so we would usually choose it only after we've exhausted our other options.

In [13]:
import pandas as pd


# Drop all rows with missing data in 'Embarked' column
print('Embarked Column')
df = pd.read_csv('data/titanic.csv')
print('Dataset before:', len(df))
print('Missing values:', df['Embarked'].isnull().sum())
df.dropna(subset=['Embarked'], how='any', inplace=True)
print('Dataset after:', len(df))
print('\n')

# Drop all rows with missing data in any column
print('Any Column')
df = pd.read_csv('data/titanic.csv')
print('Dataset before:', len(df))
df.dropna(how='any', inplace=True)
print('Dataset after:', len(df))

Embarked Column
Dataset before: 891
Missing values: 2
Dataset after: 889


Any Column
Dataset before: 891
Dataset after: 183


#### Dropping columns

<p style="line-height:1.75;font-size:16px">
If a column (feature) has many missing values it usually makes sense to drop the column. Although we could, in theory, impute the data (more on that soon) when too much of it is missing, imputation will not work well and therefore dropping the column completely is probably a better idea.<br>
It's important to note that if the column we're considering dropping is crucial for our analysis we won't be able to use this solution and instead we'll have to think of how we can fill in this data (e.g. collect more samples).

In [21]:
import pandas as pd


# Drop a column with many missing values
df = pd.read_csv('data/titanic.csv')
print('Dataset before:', len(df))
print('Missing values:', df['Cabin'].isnull().sum())
print('Missing values (%):', 100 * df['Cabin'].isnull().sum()/len(df))
df.drop('Cabin', axis=1, inplace=True)

Dataset before: 891
Missing values: 687
Missing values (%): 77.10437710437711


#### Imputation

<p style="line-height:1.75;font-size:16px">
Imputation is the process of replacing missing data with substituted values. This is usually done by looking at other non-missing values of a feature and choosing a method to infer the missing values. A common method to impute missing numerical values is to use the mean or median. In the case of categorical features, the mode is often used.<br>
Imputation introduces synthetic samples into our data and should be used with caution.

In [27]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import numpy as np

# Imputing single numerical column using mean
print('Mean')
df = pd.read_csv('data/titanic.csv')
train, test = train_test_split(df, train_size=0.8, random_state=42)
print(f'Missing Age: {train["Age"].isnull().sum()}, Average Age: {np.mean(train["Age"])}')
simple_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
train['Age'] = simple_imputer.fit_transform(train[['Age']])
print(f'Missing Age: {train["Age"].isnull().sum()}, Average Age: {np.mean(train["Age"])}')
print('\n')

# Imputing single numerical column using constant
print('Constant')
df = pd.read_csv('data/titanic.csv')
train, test = train_test_split(df, train_size=0.8, random_state=42)
print(f'Missing Age: {train["Age"].isnull().sum()}, Average Age: {np.mean(train["Age"])}')
simple_imputer = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=31)
train['Age'] = simple_imputer.fit_transform(train[['Age']])
print(f'Missing Age: {train["Age"].isnull().sum()}, Average Age: {np.mean(train["Age"])}')

Mean
Missing Age: 140, Average Age: 29.498846153846156
Missing Age: 0, Average Age: 29.498846153846152


Constant
Missing Age: 140, Average Age: 29.498846153846156
Missing Age: 0, Average Age: 29.794016853932586


In [62]:
# Imputing single categorical column using mode
print('Mean')
df = pd.read_csv('data/iowa_housing_price.csv')
train, test = train_test_split(df, train_size=0.8, random_state=42)
print(f'Missing BsmtQual: {train["BsmtQual"].isnull().sum()}, Mode Value: {train["BsmtQual"].mode().values[0]}')
simple_imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
train['BsmtQual'] = simple_imputer.fit_transform(train[['BsmtQual']])
print(f'Missing BsmtQual: {train["BsmtQual"].isnull().sum()}, Mode Value: {train["BsmtQual"].mode().values[0]}')
print('\n')

# Imputing single categorical column using constant
print('Constant')
df = pd.read_csv('data/iowa_housing_price.csv')
train, test = train_test_split(df, train_size=0.8, random_state=42)
print(f'Missing BsmtQual: {train["BsmtQual"].isnull().sum()}, Unique Values: {train["BsmtQual"].unique()}')
simple_imputer = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value='UNKNOWN')
train['BsmtQual'] = simple_imputer.fit_transform(train[['BsmtQual']])
print(f'Missing BsmtQual: {train["BsmtQual"].isnull().sum()}, Unique Values: {train["BsmtQual"].unique()}')

Mean
Missing BsmtQual: 28, Mode Value: TA
Missing BsmtQual: 0, Mode Value: TA


Constant
Missing BsmtQual: 28, Unique Values: ['TA' 'Gd' 'Fa' nan 'Ex']
Missing BsmtQual: 0, Unique Values: ['TA' 'Gd' 'Fa' 'UNKNOWN' 'Ex']


<p style="line-height:1.75;font-size:16px">
    Although the <code>SimpleImputer</code> is the one most commonly used, we could also impute values using a more complex imputer called <code>KNNImputer</code>. This imputer looks for k similar rows, without missing values for the feature we are trying to impute, and calculates the missing values as the average (or weighted average) of that feature in the most similar rows - let's see how it works

In [58]:
from sklearn.impute import KNNImputer

df = pd.read_csv('data/titanic.csv')
df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
train, test = train_test_split(df, train_size=0.8, random_state=42)
train = pd.get_dummies(train)
print(f'Missing Age: {train["Age"].isnull().sum()}, Average Age: {np.mean(train["Age"])}')
knn_imputer = KNNImputer(missing_values=np.nan, n_neighbors=5, weights='uniform')
train[train.columns] = knn_imputer.fit_transform(train)
print(f'Missing Age: {train["Age"].isnull().sum()}, Average Age: {np.mean(train["Age"])}')

Missing Age: 140, Average Age: 29.498846153846156
Missing Age: 0, Average Age: 29.805252808988765


## Handling Categorical Features

### What is it?

<p style="line-height:1.75;font-size:16px">
A categorical feature is one that only takes a finite number of distinct values. These values often have an underlying meaning and can't simply be treated as number. Moreover, in many cases, these categorical features are formatted as text.

### Why is it an issue?

<p style="line-height:1.75;font-size:16px">
We know that machine learning models work with numbers under the hood and as such they rely on various numerical properties (e.g. 4 > 3). However, with categorical variables, the numerical properties don't always hold and this might introduce noise to our data and confuse the mode. For instance, think of a categorical variable such as a zipcode - 40013 is greater than 35593 but do we want our model to treat it a such? Probably not since we know that's not how zipcodes work.<br>
    On the other hand, if we have a categorical feature such a shirt size which can be Small, Medium or Large, transforming it into an ordinal variable with the values 1, 2 and 3 might makes sense.<br>
    Categorical variables should be dealt with on a case by case basis and there isn't one solution that fits all.

### Possible Solutions

#### One Hot Encoding

In [36]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

df = pd.read_csv('data/banking.csv')
train, test = train_test_split(df, train_size=0.8, random_state=42)

print(train[['education']].head(10))
one_hot_encoder = OneHotEncoder(sparse=False)
train[one_hot_encoder.categories_[0]] = one_hot_encoder.fit_transform(train[['education']])
print(train[list(one_hot_encoder.categories_[0]) + ['education']].head(10))

                 education
12556             basic.6y
35451          high.school
30592  professional.course
17914    university.degree
3315              basic.4y
29191             basic.4y
30549             basic.9y
6098     university.degree
6252     university.degree
13373              unknown
       basic.4y  basic.6y  basic.9y  high.school  illiterate  \
12556       0.0       1.0       0.0          0.0         0.0   
35451       0.0       0.0       0.0          1.0         0.0   
30592       0.0       0.0       0.0          0.0         0.0   
17914       0.0       0.0       0.0          0.0         0.0   
3315        1.0       0.0       0.0          0.0         0.0   
29191       1.0       0.0       0.0          0.0         0.0   
30549       0.0       0.0       1.0          0.0         0.0   
6098        0.0       0.0       0.0          0.0         0.0   
6252        0.0       0.0       0.0          0.0         0.0   
13373       0.0       0.0       0.0          0.0         0.0   

#### Ordinal Encoding

In [16]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

df = pd.read_csv('data/banking.csv')
train, test = train_test_split(df, train_size=0.8, random_state=42)

print(train[['education']].head(10))
ordinal_encoder = OrdinalEncoder(categories=[['unknown', 'illiterate', 'basic.4y', 'basic.6y', 'basic.9y', 
                                             'high.school', 'professional.course','university.degree']])
train['education'] = ordinal_encoder.fit_transform(train[['education']])
print(train[['education']].head(10))

                 education
12556             basic.6y
35451          high.school
30592  professional.course
17914    university.degree
3315              basic.4y
29191             basic.4y
30549             basic.9y
6098     university.degree
6252     university.degree
13373              unknown
       education
12556        3.0
35451        5.0
30592        6.0
17914        7.0
3315         2.0
29191        2.0
30549        4.0
6098         7.0
6252         7.0
13373        0.0


In [5]:
df['education'].unique()

array(['basic.4y', 'unknown', 'university.degree', 'high.school',
       'basic.9y', 'professional.course', 'basic.6y', 'illiterate'],
      dtype=object)

#### Hash Encoder

In [37]:
!pip install -U category_encoders

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 82 kB 565 kB/s             
     |████████████████████████████████| 233 kB 30.0 MB/s            
     |████████████████████████████████| 9.9 MB 27.6 MB/s            


In [39]:
from category_encoders import HashingEncoder
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('data/banking.csv')
train, test = train_test_split(df, train_size=0.8, random_state=42)

print(train[['education']].head(10))
hashing_encoder = HashingEncoder(n_components=8)
hashing_encoder.fit_transform(train[['education']])

                 education
12556             basic.6y
35451          high.school
30592  professional.course
17914    university.degree
3315              basic.4y
29191             basic.4y
30549             basic.9y
6098     university.degree
6252     university.degree
13373              unknown


,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7
12556,0,0,0,0,0,0,1,0
35451,0,0,0,0,0,1,0,0
30592,0,0,0,1,0,0,0,0
17914,0,0,1,0,0,0,0,0
3315,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...
6265,0,0,0,0,0,1,0,0
11284,0,0,1,0,0,0,0,0
38158,0,0,0,0,0,1,0,0
860,0,0,0,1,0,0,0,0


#### Target Encoding

In [43]:
from category_encoders import TargetEncoder
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('data/banking.csv')
train, test = train_test_split(df, train_size=0.8, random_state=42)

print(train[['education']].head(10))
target_encoder = TargetEncoder()
target_encoder.fit_transform(train[['education']], train['y'])[:10]

                 education
12556             basic.6y
35451          high.school
30592  professional.course
17914    university.degree
3315              basic.4y
29191             basic.4y
30549             basic.9y
6098     university.degree
6252     university.degree
13373              unknown


,education
12556,0.081213
35451,0.107876
30592,0.112827
17914,0.136299
3315,0.103715
29191,0.103715
30549,0.078226
6098,0.136299
6252,0.136299
13373,0.141715
